In [1]:
import cfdmod.hfpi_mock as hfpi
import cfdmod.hfpi_coord as coord
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import pathlib

hfpi.set_plt_style()

### Report directory
base_base = pathlib.Path("/mnt/disk01/prd-eng/portfolio-eng/simulation_files/petra_nova_fluidmechanics/comparison/dynamic_analysis")
subproject_path = base_base / "subproj_main"
structure_path = base_base / "struct_original"
case_path = subproject_path / "out"
data_path = subproject_path / "forces/"

base_folder = pathlib.Path("/mnt/disk01/prd-eng/cases/petra_nova_fluidmechanics/engmod_case")
experimental_data_path = base_folder / "../comparison/force_by_floor/exp_csv/"



In [2]:
### building dimensions
H = 250.424
Bx = 69.14
By = 90.6
B = max(Bx, By)

dimensions = coord.DimensionSpecs(base=B, height=H)

In [3]:
modes_csv = structure_path / "modes.csv"

n_modes = 12
floors_csv = structure_path / "nodes.csv"
phi_csvs = [structure_path / f"phi_{i}.csv" for i in range(1, n_modes+1)]

struct_data = hfpi.HFPIStructuralData.build(
    modes_csv=modes_csv,
    floors_csv=floors_csv,
    phi_floors_csvs=phi_csvs,
    max_active_modes=5,
)

In [4]:
directions = ["000", "030", "060", "090", "120", "150", "180", "210", "240", "270", "300", "330"]
float_directions = [float(d) for d in directions]
recurrence_periods = [1, 10, 50]
xis = [0.01, 0.02]

In [5]:
directional_forces = {}

for d in directions:
    cf_x_h5 = data_path / f"FX_{d}.h5"
    cf_y_h5 = data_path / f"FY_{d}.h5"
    cm_z_h5 = data_path / f"MZ_{d}.h5"
    forces_data = hfpi.HFPIForcesData.build(cf_x_h5, cf_y_h5, cm_z_h5)
    directional_forces[float(d)] = forces_data

directional_forces.keys()

dict_keys([0.0, 30.0, 60.0, 90.0, 120.0, 150.0, 180.0, 210.0, 240.0, 270.0, 300.0, 330.0])

In [6]:
wind_analytical = coord.WindAnalysis(
    directional_velocity_multiplier={d: 1 for d in float_directions},
    directional_roughness_cats=pd.DataFrame(),
)

In [7]:
results_save = pathlib.Path("/mnt/dev01_fs_shared/waine/repos/cfdmod/output/hfpi_test")
analysis_full = coord.HFPIAnalysisFull(
    wind_analytics=wind_analytical,
    dimensions=dimensions,
    structural_data=struct_data,
    directional_forces=directional_forces,
    save_folder=results_save,
)

In [8]:
all_parameters = analysis_full.generate_combined_parameters(
    directions=float_directions,
    recurrence_periods=recurrence_periods,
    xis=xis,
)

analysis_full.solve_all(all_parameters, max_workers=10)

[2025-06-13 15:36:55,608] [INFO] - cfdmod - Solving HFPI for: {"direction":0.0,"xi":0.01,"recurrence_period":1.0} (hfpi_coord.py:61)
[2025-06-13 15:36:56,185] [INFO] - cfdmod - Solving HFPI for: {"direction":0.0,"xi":0.01,"recurrence_period":50.0} (hfpi_coord.py:61)
[2025-06-13 15:36:56,759] [INFO] - cfdmod - Solving HFPI for: {"direction":0.0,"xi":0.02,"recurrence_period":10.0} (hfpi_coord.py:61)
[2025-06-13 15:36:57,345] [INFO] - cfdmod - Solving HFPI for: {"direction":30.0,"xi":0.01,"recurrence_period":1.0} (hfpi_coord.py:61)
[2025-06-13 15:36:57,885] [INFO] - cfdmod - Solving HFPI for: {"direction":30.0,"xi":0.01,"recurrence_period":50.0} (hfpi_coord.py:61)
[2025-06-13 15:36:58,368] [INFO] - cfdmod - Solved HFPI in 2.76 for: {"direction":0.0,"xi":0.01,"recurrence_period":1.0}! (hfpi_coord.py:64)
[2025-06-13 15:36:58,430] [INFO] - cfdmod - Solving HFPI for: {"direction":0.0,"xi":0.01,"recurrence_period":10.0} (hfpi_coord.py:61)
[2025-06-13 15:36:58,423] [INFO] - cfdmod - Saved HFPI 